In [59]:
using LinearAlgebra
# SVD algorithm
# Takes matrix A as input, outputs unitary matrices U, V and diagonal matrix S
# Such that A = U*S*V^T

# Compact SVD, computes only the r column vectors of U and r row vectors of V
# where r is the rank of sigma
function compactSVD(A)
    m, n = size(A)
        
    if (n > m)
        throw(ArgumentError(A, "Matrix must be tall."))
    end

    # compute singular values
    sigma = eigvals(A' * A)
    
    # remove the zero singular values
    for s in sigma
        if s ≈ 0
            pop!(sigma, s)
        end
    end
    
    # calculate r
    sig_rank = size(filter((x -> x != 0), sigma), 1)
    
    #singVec_V = eigvecs(SymTridiagonal(A' * A), sigma)
    singVec_V = eigvecs(hessenberg(Symmetric(A' * A)).H, sigma)
    
    # does not work on complex eigen values
    # Calculate the squareroot of the eigenvalues of the A'A matrix, which are the eigenvalues of A
    sigma = broadcast((x -> sqrt(abs(x))), sigma)
    # Sort singVec_V and sigma so the eigenvalues are in descending order
    singVec_V = singVec_V[:, sortperm(sigma)]
    sigma = sort(sigma)

    # Compute left singular vectors
    # initializing the left singular vector
    singVec_U = Array{Float64}(undef, m, sig_rank)
        
    # Compute the left singular vector
    for i in 1:sig_rank
        # compute AV_i
        temp_A = A * singVec_V[:, i]
        # normalize AV_i
        temp_A_normal = temp_A / sigma[i]
        singVec_U[:,i] = temp_A_normal'
    end
    return singVec_U, sigma, singVec_V
end


compactSVD (generic function with 2 methods)

In [60]:
A = rand(10,10)
myF = compactSVD(A);
myU, myS, myV = myF;
F = svd(A)
U, S, V = F
A ≈ myU * Diagonal(myS) * myV'

true

In [50]:
norm(A - U * Diagonal(S) * V') / norm(A)

1.0287707396528748e-15

In [51]:
norm(A - myU * Diagonal(myS) * myV') / norm(A)

2.7351586850221313e-16

In [44]:
norm(myU' * myU - I)

3.3739071121393184e-15

In [45]:
norm(U' * U - I)

9.823215122697685e-16

In [46]:
norm(myV' * myV - I)

1.5008406958793157e-15